A Markov Decision Process (MDP) is a fully observable, probabilistic state model. The most common formulation of MDPs is a Discounted-Reward Markov Decision Process. A discount-reward MDP is a tuple
 containing:

a state space
;

initial state
;

actions
 applicable in each state
 that our agent can execute;

transition probabilities
 for
 and
;

rewards
 positive or negative of transitioning from state
 to state
 using action
; and

a discount factor
.
A multi-armed bandit (also known as an
-armed bandit) is defined by a set of random variables
 where:

, such that
 is the arm of the bandit; and

 the index of the play of arm
;

Successive plays
 are assumed to be independently distributed, but we do not know the probability distributions of the random variables.

The idea is that a gambler iteratively plays rounds, observing the reward from the arm after each round, and can adjust their strategy each time. The aim is to maximise the sum of the rewards collected over all rounds.

Multi-arm bandit strategies aim to learn a policy
, where
 is the play.

 Given that we do not know the probability distributions, a simple strategy is simply to select the arm given a uniform distribution; that is, select each arm with the same probability. This is just uniform sampling.

Then, the Q-value for an action
 can be estimated using the following formula:



where
 is the number of rounds so far,
 is the number of times
 selected in previous rounds, and
 is the reward obtained in the
-th round for playing arm
..

The idea here is that for a multi-armed bandit problem, we explore the options uniformly for some time, and then once we are confident we have enough samples (when the changes to the values of
 start to stabilise), we start selecting
. This is known as the
-first strategy, where the parameter
 (epsilon), determines how many rounds to select random actions before moving to the greedy action.

But what is the issue? Time is wasted equally in all actions using the uniform distribution. Instead, we can focus on the most promising actions given the rewards we have received so far.

In [ ]:
from collections import defaultdict
import random
from qtable import QTable


""" Run a bandit algorithm for a number of episodes, with each episode
being a set length.
"""

def run_bandit(bandit, episodes=200, episode_length=500, drift=True):

    # The actions available
    actions = [0, 1, 2, 3, 4]

    # A dummy state
    state = 1

    rewards = []
    for _ in range(0, episodes):
        bandit.reset()

        # The probability of receiving a payoff of 1 for each action
        probabilities = [0.1, 0.3, 0.7, 0.2, 0.1]

        # The number of times each arm has been selected
        times_selected = defaultdict(lambda: 0)
        qtable = QTable()

        episode_rewards = []
        for step in range(0, episode_length):

            # Halfway through the episode, change the probabilities
            if drift and step == episode_length / 2:
                probabilities = [0.5, 0.2, 0.0, 0.3, 0.3]

            # Select an action using the bandit
            action = bandit.select(state, actions, qtable)

            # Get the reward for that action
            reward = 0
            if random.random() < probabilities[action]:
                reward = 5

            episode_rewards += [reward]

            times_selected[action] = times_selected[action] + 1
            qtable.update(
                state,
                action,
                (reward / times_selected[action])
                - (qtable.get_q_value(state, action) / times_selected[action]),
            )

        rewards += [episode_rewards]

    return rewards

monte carlo

In [ ]:
import sys
import gym
import numpy as np
from collections import defaultdict

from plot_utils import plot_blackjack_values, plot_policy
env = gym.make('Blackjack-v0')

print(env.observation_space)
print(env.action_space)

for i_episode in range(3):
    state = env.reset()
    while True:
        print(state)
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        if done:
            print('End game! Reward: ', reward)
            print('You won :)\n') if reward > 0 else print('You lost :(\n')
            break

def generate_episode_from_limit_stochastic(bj_env):
    episode = []
    state = bj_env.reset()
    while True:
        probs = [0.8, 0.2] if state[0] > 18 else [0.2, 0.8]
        action = np.random.choice(np.arange(2), p=probs)
        next_state, reward, done, info = bj_env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

for i in range(3):
    print(generate_episode_from_limit_stochastic(env))

def mc_prediction_q(env, num_episodes, generate_episode, gamma=1.0):
    # initialize empty dictionaries of arrays
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        # generate an episode
        episode = generate_episode(env)
        # obtain the states, actions, and rewards
        states, actions, rewards = zip(*episode)
        # prepare for discounting
        discounts = np.array([gamma**i for i in range(len(rewards)+1)])
        # update the sum of the returns, number of visits, and action-value
        # function estimates for each state-action pair in the episode
        for i, state in enumerate(states):
            returns_sum[state][actions[i]] += sum(rewards[i:]*discounts[:-(1+i)])
            N[state][actions[i]] += 1.0
            Q[state][actions[i]] = returns_sum[state][actions[i]] / N[state][actions[i]]
    return Q


# obtain the action-value function
Q = mc_prediction_q(env, 500000, generate_episode_from_limit_stochastic)

# obtain the corresponding state-value function
V_to_plot = dict((k,(k[0]>18)*(np.dot([0.8, 0.2],v)) + (k[0]<=18)*(np.dot([0.2, 0.8],v))) \
         for k, v in Q.items())

# plot the state-value function
plot_blackjack_values(V_to_plot)



Dynamic programming

In [ ]:
import numpy as np
import copy

import check_test
from frozenlake import FrozenLakeEnv
from plot_utils import plot_values

env = FrozenLakeEnv()

# print the state space and action space
print(env.observation_space)
print(env.action_space)

# print the total number of states and actions
print(env.nS)
print(env.nA)

env.P[1][0]
def policy_evaluation(env, policy, gamma=1, theta=1e-8):
    V = np.zeros(env.nS)

    ## TODO: complete the function

    return V
random_policy = np.ones([env.nS, env.nA]) / env.nA

# evaluate the policy
V = policy_evaluation(env, random_policy)

plot_values(V)


check_test.run_check('policy_evaluation_check', policy_evaluation)



In [ ]:
import numpy as np
import os
import inspect
%matplotlib inline
import math
import time
from IPython.display import clear_output
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir)

from utils import plotting
class gridworld:
    def __init__(self,dims,movementReward,terminalStates,gamma=1):
        self.dims = dims
        self.movementReward = movementReward
        self.terminalStates = terminalStates
        self.stateValues = np.zeros([dims,dims])
        self.gamma = gamma

class policyEval(gridworld):
    def __init__(self,prob=[.25,.25,.25,.25],dims=4,movementReward=-1,terminalStates = [[0,0],[3,3]]):
        assert not any(n < 0 for n in prob), 'probabilities are always non-negative dammit!'
        assert np.sum(prob)==1,'sum is not 1'

        super(policyEval,self).__init__(dims,movementReward,terminalStates)
        #self.currentValues = np.zeros([dims,dims])
        self.action_prob = {'UP':prob[0],'DOWN':prob[1],'LEFT':prob[2],'RIGHT':prob[3]}



    def getNextState(self,currentState,action,dims):
        if action == 'UP':
            nextState = (currentState[0]-1,currentState[1])
        if action == 'DOWN':
            nextState = (currentState[0]+1,currentState[1])
        if action == 'LEFT':
            nextState = (currentState[0],currentState[1]-1)
        if action == 'RIGHT':
            nextState = (currentState[0],currentState[1]+1)
        nextState = (max(nextState[0],0),max(nextState[1],0))
        nextState = (min(nextState[0],dims-1),min(nextState[1],dims-1))
        return nextState


    def evaluate(self):
        theta = 1e-4

        while(1):
            for i in range(self.dims):
                for j in range(self.dims):
                    if [i,j] not in self.terminalStates:
                        newValue = 0
                        for action in self.action_prob.keys():
                            nextState = self.getNextState((i,j),action,self.dims)
                            newValue += self.action_prob[action]*(self.movementReward + self.gamma*self.stateValues[nextState[0]][nextState[1]])

                        if abs(self.stateValues[i][j] - newValue) < theta:
                            self.stateValues[i][j] = newValue
                            return self.stateValues
                        self.stateValues[i][j] = newValue



uniformDistributionPolicy = policyEval()
plotting.plot_gridworld_values(uniformDistributionPolicy.evaluate())



In [ ]:
q learning

In [ ]:
# Adapted from: https://github.com/lazyprogrammer/machine_learning_examples/tree/master/rl
# the Q-Learning method to find the optimal policy and value function
import numpy as np
import matplotlib.pyplot as plt
from gridWorldGame import standard_grid, negative_grid,print_values, print_policy

SMALL_ENOUGH = 1e-3
GAMMA = 0.9
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')
ALPHA = 0.1

def max_dict(d):
  # returns the argmax (key) and max (value) from a dictionary
  max_key = None
  max_val = float('-inf')
  for k, v in d.items():
    if v > max_val:
      max_val = v
      max_key = k
  return max_key, max_val

def random_action(a, eps=0.1):
  # epsilon-soft to ensure all states are visited
  p = np.random.random()
  if p < (1 - eps):
    return a
  else:
    return np.random.choice(ALL_POSSIBLE_ACTIONS)

grid = negative_grid(step_cost=-0.1)

# print rewards
print("rewards:")
print_values(grid.rewards, grid)

# no policy initialization, policy is derived from most recent Q like SARSA

# initialize Q(s,a)
Q = {}
states = grid.all_states()
for s in states:
  Q[s] = {}
  for a in ALL_POSSIBLE_ACTIONS:
    Q[s][a] = 0

# initial Q values for all states in grid
print(Q)

update_counts = {}
update_counts_sa = {}
for s in states:
  update_counts_sa[s] = {}
  for a in ALL_POSSIBLE_ACTIONS:
    update_counts_sa[s][a] = 1.0

# repeat until convergence
t = 1.0
deltas = []
for it in range(10000):
  if it % 100 == 0:
    t += 1e-2
  if it % 2000 == 0:
    print("iteration:", it)

  # instead of 'generating' an epsiode, we will PLAY
  # an episode within this loop
  s = (2, 0) # start state
  grid.set_state(s)

  # the first (s, r) tuple is the state we start in and 0
  # (since we don't get a reward) for simply starting the game
  # the last (s, r) tuple is the terminal state and the final reward
  # the value for the terminal state is by definition 0, so we don't
  # care about updating it.
  a, _ = max_dict(Q[s])
  biggest_change = 0
  while not grid.game_over():
    a = random_action(a, eps=0.5/t) # epsilon-greedy
    # random action also works, but slower since you can bump into walls
    # a = np.random.choice(ALL_POSSIBLE_ACTIONS)
    r = grid.move(a)
    s2 = grid.current_state()

    # adaptive learning rate
    alpha = ALPHA / update_counts_sa[s][a]
    update_counts_sa[s][a] += 0.005

    # we will update Q(s,a) AS we experience the episode
    old_qsa = Q[s][a]
    # the difference between SARSA and Q-Learning is with Q-Learning
    # we will use this max[a']{ Q(s',a')} in our update
    # even if we do not end up taking this action in the next step
    a2, max_q_s2a2 = max_dict(Q[s2])
    Q[s][a] = Q[s][a] + alpha*(r + GAMMA*max_q_s2a2 - Q[s][a])
    biggest_change = max(biggest_change, np.abs(old_qsa - Q[s][a]))

    # we would like to know how often Q(s) has been updated too
    update_counts[s] = update_counts.get(s,0) + 1

    # next state becomes current state
    s = s2
    a = a2

  deltas.append(biggest_change)



In [ ]:
# prompt: generate theory on q learning

# Theory on Q-Learning

# Q-learning is a model-free reinforcement learning algorithm that learns an optimal policy for an agent interacting with an environment.  It does so by estimating the optimal action-value function, Q*(s, a), which represents the expected cumulative discounted reward for taking action 'a' in state 's' and following the optimal policy thereafter.

# Key Concepts:

# 1. State-Action Value Function (Q-value):  Q(s, a) estimates the long-term value of taking action 'a' in state 's'.  The goal of Q-learning is to approximate Q*(s, a).

# 2. Exploration vs. Exploitation: Q-learning must balance exploration (trying out new actions) and exploitation (choosing the action with the highest estimated Q-value).  Common methods for balancing these include epsilon-greedy strategies.

# 3. Temporal Difference (TD) Learning: Q-learning updates Q-values based on observed transitions and rewards.  It uses a TD target, which is an estimate of the optimal Q-value based on the current Q-value and the reward received for taking an action. The difference between the current Q-value and the TD target is the TD error.

# 4. Update Rule: The core of Q-learning is the update rule, which iteratively refines the Q-value estimates. The update rule typically follows this form:

#    Q(s, a) = Q(s, a) + alpha * [r + gamma * max_a' Q(s', a') - Q(s, a)]

#    where:
#       - alpha is the learning rate, controlling the step size of the update.
#       - r is the immediate reward received for taking action 'a' in state 's'.
#       - gamma is the discount factor, determining the importance of future rewards.
#       - s' is the next state after taking action 'a' in state 's'.
#       - max_a' Q(s', a') is the maximum estimated Q-value for the next state s', across all possible actions a'.  This is the crucial difference between SARSA and Q-Learning.

# 5. Off-policy Learning: Q-learning is an off-policy algorithm, meaning that it learns about the optimal policy while following a different (e.g., epsilon-greedy) behavior policy. This allows it to explore more effectively.  SARSA is on-policy.

# 6. Convergence: Under certain conditions (e.g., appropriate learning rate, sufficient exploration), Q-learning converges to the optimal Q-value function Q*(s, a).


# Differences between Q-learning and SARSA:

#  - Q-learning uses the maximum estimated Q-value of the *next* state (max_a' Q(s', a')) in its update rule.
#  - SARSA uses the Q-value of the *actual* action taken in the *next* state.  SARSA is on-policy.
#  - As a result, Q-learning is more optimistic and tends to explore more.


# Example Implementation Notes from the Provided Code:

# - The code demonstrates Q-learning applied to a grid world environment and a Blackjack environment.
# - It uses an epsilon-greedy policy for action selection.
# - The learning rate is adapted based on the number of visits to each state-action pair.
# - The code visualizes the learned value function and/or policy.

# Further improvements:

# - Experience Replay
# - Prioritized Experience Replay
# - Double Q-Learning
# - Dueling Deep Q Networks
